In [ ]:
import sys
import csv
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xarray as xr

import matplotlib.colors as mcolors
import geopandas as gpd
from shapely.geometry import Point, box

import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import shapely.geometry

import holoviews as hv
hv.extension('bokeh')

import bokeh.io
bokeh.io.output_notebook()


In [ ]:
sys.path.append('../../software_modules/')
import integ_hemco_emissions

The footprint area box you are using for STILT is 
-110
-90
20
50
xres = .1, yres = .1, 4km/111 = units of degree
Remember that years of study are from 2017 - 2021 during june 1 - oct 1. I got all the well data from https://www.fractracker.org/map/us/

# Try analyzing with Geopandas

In [ ]:
arkog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/AR_OG_AllWells_05032021/ArkansasOilandGasWells_05032021.shp")

In [ ]:
colog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/CO_OilandGas_WELLS_SHP_110321/Wells.shp")

In [ ]:
kansog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/KS_SWD_WELLS_Volumes_End_2018_With_Excel_KML/KS_SWD_WELLS_Volumes_End_2018.shp")

In [ ]:
missog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/MO_OG_AllWells_04142021/MissouriWells.shp")

In [ ]:
newmexog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/NM_03152022_OGWells/WellsGIS/WellsGIS.shp")


In [ ]:
oklaog = gpd.read_file("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/OK_ClassII_2011_2018_Volumes_WithExcel/OK_ClassII_2011_2018_Volumes.shp")

In [ ]:
texasog = pd.read_csv("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/TX_Prod5-8-2021/TX_OGwells_05082021.csv")

In [ ]:
texasog.head()

In [ ]:
# the text file that came with the data says these are in North American Datum 1983 ...maybe??
# It says that nad27 were let go since nad83 were available, but it doesn't say whether they changed the units 
# to wgs for this dataset. I am assuming the later because when I build the geopandas using wgs, it keeps more of the 
# trailiing decimal points in the lat/lon. 
# Used the following site to get the reference codes: https://spatialreference.org, 
# where NAD1983 is here https://spatialreference.org/ref/epsg/2239/proj4js/
# crs="EPSG:2239" and wgs is 4326


texasog_gdf = gpd.GeoDataFrame(texasog, 
            geometry=gpd.points_from_xy(texasog.LONG83, texasog.LAT83),
            crs="EPSG:4326")
texasog_gdf.head()

In [ ]:
texasog_gdf.crs

The oklahoma data has weird lat/lon values equal to 0, so I am removing those. 
ark, colog, kansog, missog, newmexlog and texas looks fine . 


In [ ]:
oklaog.drop(np.where((oklaog.Latitude.values > 50) | (oklaog.Latitude.values < 20) | (oklaog.Longitude.values >= 0))[0], 
            inplace=True)



In [ ]:
# from bokeh.sampledata.airport_routes import airports

In [ ]:
# the crs tells the projection
# note that the below thing is needed bc geopandas needs lat,lon in that order instead of lon,lat, which is what the 
# orig data order is in

In [ ]:
# geodata = pd.DataFrame(
#     [
#         [-88.355555, 30.757778],
#         [-120.849722, 46.041111],
#         [-113.8875, 8.12],
#         [-173.24, 38.54],
#         [-85.663611, 46.154444],
#         [-98.3555, -119.1342],
#         [-9.5932, -11.2836],
#         [-3.2948, 38.2224],
#         [36.2327, 29.3626],
#         [3.3483, 47.5047],
#     ],
#     columns=["Longitude", "Latitude"],
# )

# data_gdf = gpd.GeoDataFrame(
#     geodata, geometry=gpd.points_from_xy(geodata["Longitude"], geodata["Latitude"])
# )
# data_gdf.plot()

In [ ]:

# idx = pd.date_range('1/1/2000', periods=1000)
# df  = pd.DataFrame(np.random.randn(1000, 4), index=idx, columns=list('ABCD')).cumsum()
# df.hvplot()

In [ ]:
# oklaog.hvplot.points('Longitude', 'Latitude', geo=True, color='red', alpha=0.2,
#                        xlim=(-180, -30), ylim=(20, 50), tiles='ESRI')

### Put all wells onto a single geopandas frame
Not all of the states have lat/lon as variables, but all of them do have geometry in lat/lon. Wgs84 is the most accurate and utilizes gps. The second most accurate spatial reference is nad83, followed by nad27. 
Checked here for the number to change to Wgs84 https://spatialreference.org. The number to change to wgs84 is 4326. I tried this and it worked, but for texas all of the points became identical when I had texas geopandas dataframe built in nad83. I then tried switching to Nad83 (num is 2239) for all states, and only texas plotted while all the other states didn't plot. When I did that, akansas data didn't change from units of meters to lat/lon whereas when I used the wgs84 it changed to lat/lon. I decided to create texas geopandas using wgs84 and switch all states to wgs84 and that worked. 

In [ ]:
arkog = arkog.to_crs(4326)
colog = colog.to_crs(4326)
kansog = kansog.to_crs(4326)
missog = missog.to_crs(4326)
newmexog = newmexog.to_crs(4326)
oklaog = oklaog.to_crs(4326)
# texasog_gdf = texasog_gdf.to_crs(2239) # know for sure it is already in this format

In [ ]:
allwells_gdf = gpd.GeoDataFrame(pd.concat([arkog['geometry'], colog['geometry'], kansog['geometry'], missog['geometry'],
                                          newmexog['geometry'], oklaog['geometry'], texasog_gdf['geometry']], 
                                          ignore_index=True), crs=arkog.crs)

In [ ]:
allwells_gdf

In [ ]:
ax = allwells_gdf.plot(markersize=.1, figsize=(12, 8))

xlim = ([-115, -50])
ylim = ([25, 50])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# plt.autoscale(False)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world[world.name == "United States of America"]
usa.to_crs(allwells_gdf.crs).plot(ax=ax, color='none', edgecolor='black')

In [ ]:
allwells_gdf.total_bounds

### Get the EPA .1 x .1 US emissions grid as a starting point
From Maasakkers 2016 paper. They have daily and monthly too but I just want to use this for the lat/lon grid 

In [ ]:
epagrid = xr.open_dataset('/Users/arianatribby/git/oklahoma_propane/data/EPA_methane_gridded/GEPA_Annual.nc')

In [ ]:
len(epagrid.lat.values)

In [ ]:
len(epagrid.lon.values)

In [ ]:
epadf = epagrid['emissions_1B2a_Petroleum'].to_dataframe().reset_index()

### Grid the wells onto .1 x .1 
Using https://james-brennan.github.io/posts/fast_gridding_geopandas/  as reference

In [ ]:
p1byp1_grid_gdf = gpd.GeoDataFrame(epadf,
                                   geometry=gpd.points_from_xy(epadf.lon, epadf.lat),
            crs="EPSG:4326")
p1byp1_grid_gdf.head()

In [ ]:
ax = p1byp1_grid_gdf.plot()
xlim = ([-107.5, -105.0])
ylim = ([28, 30])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

figure out how to make this mesh with just the vertices. 

#### Method 1, making cells from scratch using the bounds of the wellhead gdf 
Decided to use Method 2 below. 

In [ ]:
# xmin, ymin, xmax, ymax= allwells_gdf.total_bounds

In [ ]:
# print(xmin)
# print(xmax)
# print(ymin)
# print(ymax)

In [ ]:
# this number of cell size gives around .1 distance between lati and lati+1 which is what the epa grid is 
# 19.8745/197

In [ ]:
# n_cells=197
# cell_size = (xmax-xmin)/n_cells
# # projection of the grid
# crs = "EPSG:4326"
# # create the cells in a loop
# grid_cells = []
# for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
#     for y0 in np.arange(ymin, ymax+cell_size, cell_size):
#         # bounds
#         x1 = x0-cell_size
#         y1 = y0+cell_size
#         grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
# cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
#                                  crs=crs)

In the figure below (that is commented), you can see that the uncertainty adds up. 



In [ ]:
# fig, ax = plt.subplots(figsize = (10,10)) 
# xlim = ([-107.5, -105.0])
# ylim = ([28, 30])
# ax.set_xlim(xlim)
# ax.set_ylim(ylim)
# cell.plot(facecolor="none",ax=ax)
# p1byp1_grid_gdf.plot(ax=ax)


In [ ]:
# how many cells across and down
# n_cells=30
# cell_size = (xmax-xmin)/n_cells
# # projection of the grid
# crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
# # create the cells in a loop
# grid_cells = []
# for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
#     for y0 in np.arange(ymin, ymax+cell_size, cell_size):
#         # bounds
#         x1 = x0-cell_size
#         y1 = y0+cell_size
#         grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
# cell2 = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
#                                  crs=crs)

In [ ]:
# cell2

In [ ]:
# cell2.loc[[0],'geometry']

#### Method 2: Create grid from the EPA lat/lon points (ie draw a box from them)
But you don't need the entire US grid, cut off at the wellhead gdf endpoints

In [ ]:
xmin, ymin, xmax, ymax= allwells_gdf.total_bounds

print(ymin)
print(ymax)
print(xmin)
print(xmax)

In [ ]:
p1byp1_lat = epagrid.lat.values
p1byp1_lon = epagrid.lon.values


In [ ]:
p1byp1_lat = p1byp1_lat[(p1byp1_lat >= ymin) | (p1byp1_lat <= ymax)]

In [ ]:
p1byp1_lon = p1byp1_lon[(p1byp1_lon >= xmin) | (p1byp1_lon <= xmax)]

In [ ]:
crs = "EPSG:4326"

grid_cells = []
for i in range(len(p1byp1_lon)-1):
    for j in range(len(p1byp1_lat)-1):
        x0 = p1byp1_lon[i]
        x1 = p1byp1_lon[i+1]
        y0 = p1byp1_lat[j]
        y1 = p1byp1_lat[j+1]
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1))
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=crs)


In [ ]:
cell.tail()

In [ ]:
print(len(cell))

In [ ]:
fig, ax = plt.subplots(figsize = (5,5)) 
# zooming in to a small section of the grid you made!
xlim = ([-107.5, -105.0])
ylim = ([28, 30])
ax.set_xlim(xlim)
ax.set_ylim(ylim)
cell.plot(facecolor="none",ax=ax)
p1byp1_grid_gdf.plot(ax=ax)


In [ ]:
cell.loc[[0],'geometry']

#### Get density of wells per .1x.1 square
Using this as a reference https://james-brennan.github.io/posts/fast_gridding_geopandas/ 

Merge the cell with the wellhead points according to geometries in cell (within the cell) 

In [ ]:
# merged = gpd.sjoin(allwells_gdf, cell, how='left', op='within')
merged = gpd.sjoin(allwells_gdf, cell, how='left', predicate='within')

In [ ]:
merged

In [ ]:
print(len(merged))
print(len(allwells_gdf))


Now count the number of wells within each grid cell. Aggregate the merged so the number of rows for each grid cell is counted. This aggregation is called "dissolving" in geopandas (making data more sparse). 

In [ ]:
# make a simple count variable that we can sum
merged['n_wells']=1
# Compute stats per grid cell -- aggregate wells to grid cells with dissolve
dissolved_aggregation = merged.dissolve(by="index_right", aggfunc="count")


In [ ]:
print(len(dissolved_aggregation))
dissolved_aggregation

In [ ]:
dissolved_aggregation.index

In [ ]:
max(dissolved_aggregation.index) - min(dissolved_aggregation.index)

In [ ]:
len(dissolved_aggregation)

How did the loc work? 

In [ ]:
# put this into cell
cell.loc[dissolved_aggregation.index, 'n_wells'] = dissolved_aggregation.n_wells.values

In [ ]:
print(len(cell))
max(dissolved_aggregation.n_wells)

In [ ]:
dissolved_aggregation.index.values

In [ ]:
cell.loc[dissolved_aggregation.index.values]

In [ ]:
cell.loc[dissolved_aggregation.index]

In [ ]:
fig = plt.figure()
ax = cell.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)

xlim = ([-107.5, -105.0])
ylim = ([28, 30])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

cell.plot(facecolor="none",ax=ax)
p1byp1_grid_gdf.plot(ax=ax)

plt.show()

In [ ]:
fig = plt.figure()
# ax = dissolved_aggregation.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)
ax = cell.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)


xlim = ([-115, -80])
ylim = ([25, 50])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world[world.name == "United States of America"]
usa.to_crs(allwells_gdf.crs).plot(ax=ax, color='none', edgecolor='black')
# fig.savefig('well_density.png')

plt.savefig("well_density.png")
plt.show()

Plot the grid on top? How do you know that they fit in each cell? 

### Get your revised emissions for ethane, methane, propane 
The commented stuff is a test before you did it for all years. It was using old unscaled hemco emissions

In [ ]:
gcemiss = xr.open_dataset('/Users/arianatribby/git/propane_bayes_local/data/HEMCO_output/HEMCO_diagnostics.201607010000.nc.mp')



In [ ]:
# gcemiss.lat

In [ ]:
lat_fromgcemiss_ofinterest = gcemiss.lat.values
lon_fromgcemiss_ofinterest = gcemiss.lon.values

Compare this to the well area 

In [ ]:
xmin, ymin, xmax, ymax= allwells_gdf.total_bounds

In [ ]:
print(xmin)
print(xmax)
print(ymin)
print(ymax)

In [ ]:
# list(lat_fromgcemiss_ofinterest[(lat_fromgcemiss_ofinterest >= 20) & (lat_fromgcemiss_ofinterest <= 42)])

In [ ]:
# list(lon_fromgcemiss_ofinterest[(lon_fromgcemiss_ofinterest >= -110) & (lon_fromgcemiss_ofinterest <= -85)])

In [ ]:
gcC3H8_footprintregion = gcemiss.sel(lat=list(lat_fromgcemiss_ofinterest[(lat_fromgcemiss_ofinterest >= 20) & (lat_fromgcemiss_ofinterest <= 42)]), 
                            lon=list(lon_fromgcemiss_ofinterest[(lon_fromgcemiss_ofinterest >= -110) & (lon_fromgcemiss_ofinterest <= -85)]))

In [ ]:
gcC3H8_integ_072016 = integ_hemco_emissions.integ(gcC3H8_footprintregion,'EmisC3H8_Anthro',1)

In [ ]:
time = 60*60*24*365

In [ ]:
print(gcC3H8_integ_072016*time*1e-9)
print(gcC3H8_integ_072016*time*1e-9*1.3)



and that is Tg/yr manually scaled since these are old, unscaled emiss. This is not methane! This is propane. C3 to C1 is .17 from previous paper. Using this emission factor, this implies 3.18 Tg/yr methane from this region. This is reasonable (perhaps underestimating slightly) since there was that paper that estimated permian emits 3 Tg/yr methane. 

## Emissions based on Permian Basin

2020 satellite study found Permian emits 2.7 Tg/year. Paul suggests to get an emission rate for all wells that is the same based on the permian only. they use the domain of 29-34 lat, 100-106W lon. 

Zhang, Y. et al. Quantifying methane emissions from the largest oil-producing basin in the United States from space. Sci. Adv. 6, eaaz5120 (2020).

In [ ]:
print(allwells_gdf.total_bounds)

In [ ]:
allwells_gdf.cx[-106:-100, 29:34].total_bounds

In [ ]:
# try using cx 
permianwells = allwells_gdf.cx[-106:-100, 29:34]

In [ ]:
dissolved_aggregation.total_bounds

In [ ]:
dissolved_aggregation.cx[-106:-100, 29:34].total_bounds

In [ ]:
permianwell_density = dissolved_aggregation.cx[-106:-100, 29:34]

Zhang et al esimated 2.7 +/- 0.5 Tg/year for permian emissions. 

Going to adopt the error of 0.5 Tg/yr as in Zhang et al. for the methane above. 

Assuming a normal distribution for both emissions and emission ratios, combining the error via error propagation for the C3 and C2 emissions. 

Here are the NOAA ratios after doing bootstrap (Figure S21, the code is in 

git/Peer comments review C3 bayes paper/Plots_NOAAbootstrpGeophserr.ipynb

```
[0.1653053 0.1687432] c3/c1 2017 SGP masss/mass 95% ci
[0.16165111 0.16478708] C2/C1 2017 SGP mass/mass 95% ci 

0.0017302837807093435 c3/c1 2017 standard deviation SGP masss/mass
0.0015725221005948586 C2/C1 2017 standard deviation SGP mass/mass

0.16696523848130124 c3/c1 2017 mean SGP masss/mass
0.16335588804346804 C2/C1 2017 mean SGP mass/mass
```


In [ ]:
# tot err = sqrt( (std/mean ratio)**2 + (emiss err/mean zhang emiss)**2 )
propagated_err_for_c3 = np.sqrt((0.0017302837807093435/0.16696523848130124)**2 + (0.5/2.7)**2)
propagated_err_for_c2 = np.sqrt((0.0015725221005948586/0.16335588804346804)**2 + (0.5/2.7)**2)

In [ ]:
permianTgyr_propane = (2.7)*0.16696523848130124
permianTgyr_ethane = (2.7)*0.16335588804346804

In [ ]:
print('2.7', '+/', '0.5', 'Tg/s')
print(permianTgyr_ethane, '+/-', propagated_err_for_c2, 'Tg/s')
print(permianTgyr_propane, '+/-', propagated_err_for_c3, 'Tg/s')

Get the emissions per well (and the error per well)

In [ ]:
permianTgyr_methaneperwell = 2.7/sum(permianwell_density.n_wells.values)
permianTgyr_methaneperwell_error = 0.5/sum(permianwell_density.n_wells.values)

permianTgyr_propaneperwell = permianTgyr_propane/sum(permianwell_density.n_wells.values)
permianTgyr_propaneperwell_error = propagated_err_for_c3/sum(permianwell_density.n_wells.values)

permianTgyr_ethaneperwell = permianTgyr_ethane/sum(permianwell_density.n_wells.values)
permianTgyr_ethaneperwell_error = propagated_err_for_c2/sum(permianwell_density.n_wells.values)

print(permianTgyr_methaneperwell, '+/', permianTgyr_methaneperwell_error, 'Tg/yr')
print(permianTgyr_propaneperwell, '+/-', permianTgyr_propaneperwell_error, 'Tg/yr')
print(permianTgyr_ethaneperwell, '+/-', permianTgyr_ethaneperwell_error, 'Tg/yr')


Convert this to kg/s to be relevant for the inversion. 

In [ ]:
permiankgs_methaneperwell = permianTgyr_methaneperwell*1e9/time
permiankgs_methaneperwell_error = permianTgyr_methaneperwell_error*1e9/time

permiankgs_propaneperwell = permianTgyr_propaneperwell*1e9/time
permiankgs_propaneperwell_error = permianTgyr_propaneperwell_error*1e9/time

permiankgs_ethaneperwell = permianTgyr_ethaneperwell*1e9/time
permiankgs_ethaneperwell_error = permianTgyr_ethaneperwell_error*1e9/time


In [ ]:
cell['ch4_emiss_kgs'] = cell['n_wells']*permiankgs_methaneperwell
cell['ch4_emiss_kgs_unc'] = cell['n_wells']*permiankgs_methaneperwell_error

cell['c3h8_emiss_kgs'] = cell['n_wells']*permiankgs_propaneperwell
cell['c3h8_emiss_kgs_unc'] = cell['n_wells']*permiankgs_propaneperwell_error

cell['c2h6_emiss_kgs'] = cell['n_wells']*permiankgs_ethaneperwell
cell['c2h6_emiss_kgs_unc'] = cell['n_wells']*permiankgs_ethaneperwell_error

In [ ]:
print('number of wells in footprint area:', cell['n_wells'].sum())
print('CH4 kg per second rate footprint area:', cell['ch4_emiss_kgs'].sum())
print('CH4 Tg/yr footprint area:', cell['ch4_emiss_kgs'].sum()*1e-9*time)
print('CH4 Tg/yr uncertainty footprint area:', cell['ch4_emiss_kgs_unc'].sum()*1e-9*time)

print('C3H8 Tg/yr footprint area:', cell['c3h8_emiss_kgs'].sum()*1e-9*time)
print('C3H8 Tg/yr uncertainty footprint area:', cell['c3h8_emiss_kgs_unc'].sum()*1e-9*time)

print('C2H6 Tg/yr footprint area:', cell['c2h6_emiss_kgs'].sum()*1e-9*time)
print('C2H6 Tg/yr uncertainty footprint area:', cell['c2h6_emiss_kgs_unc'].sum()*1e-9*time)


In [ ]:
explodeddf = cell.explode(index_parts=True)

In [ ]:
explodeddf.head()

In [ ]:
cell.to_file('/Users/arianatribby/git/oklahoma_propane/data/processing/wellhead_emiss_for_footprint.shp')

check that it worked 

In [ ]:
emisscheck = gpd.read_file("/Users/arianatribby/git/oklahoma_propane/data/processing/wellhead_emiss_for_footprint.shp")


In [ ]:
emisscheck.columns

In [ ]:
np.nanmax(emisscheck['ch4_emiss_'])

Got the US state shp file from https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2022&layergroup=States+%28and+equivalent%29 

In [ ]:
us_states = gpd.read_file("/Users/arianatribby/git/oklahoma_propane/data/processing/tl_2022_us_state/tl_2022_us_state.shp")
us_states = us_states.to_crs("EPSG:4326")

In [ ]:
non_continental = ['HI','VI','MP','GU','AK','AS','PR']
us49 = us_states
for n in non_continental:
    us49 = us49[us49.STUSPS != n]

In [ ]:
fig = plt.figure()
# ax = dissolved_aggregation.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)
ax = emisscheck.plot(column='ch4_emiss_', figsize=(12, 8), cmap='viridis', legend=True, 
                     legend_kwds={'label':'CH4 kg/s'})


xlim = ([-115, -80])
ylim = ([25, 50])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

us49.boundary.plot(ax=ax, edgecolor='black')

# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# usa = world[world.name == "United States of America"]
# usa.to_crs(allwells_gdf.crs).plot(ax=ax, color='none', edgecolor='black')
plt.savefig('ch4_emiss_kgpers.png')

plt.show()

In [ ]:
fig = plt.figure()
# ax = dissolved_aggregation.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)
ax = emisscheck.plot(column='c2h6_emiss', figsize=(12, 8), cmap='viridis', legend=True, 
                     legend_kwds={'label':'C2H6 kg/s'})


xlim = ([-115, -80])
ylim = ([25, 50])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

us49.boundary.plot(ax=ax, edgecolor='black')

# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# usa = world[world.name == "United States of America"]
# usa.to_crs(allwells_gdf.crs).plot(ax=ax, color='none', edgecolor='black')
plt.savefig('c2h6_emiss_kgpers.png')

plt.show()

In [ ]:
fig = plt.figure()
# ax = dissolved_aggregation.plot(column='n_wells', figsize=(12, 8), cmap='viridis', vmax=3000, legend=True)
ax = emisscheck.plot(column='c3h8_emiss', figsize=(12, 8), cmap='viridis', legend=True, 
                     legend_kwds={'label':'C3H8 kg/s'})


xlim = ([-115, -80])
ylim = ([25, 50])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

us49.boundary.plot(ax=ax, edgecolor='black')

# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# usa = world[world.name == "United States of America"]
# usa.to_crs(allwells_gdf.crs).plot(ax=ax, color='none', edgecolor='black')
plt.savefig('c3h8_emiss_kgpers.png')

plt.show()

Need to get the lat/lon grid into an xarray from this. Going to use the following:

https://cf-xarray.readthedocs.io/en/latest/generated/cf_xarray.shapely_to_cf.html#cf_xarray.shapely_to_cf
https://github.com/xarray-contrib/cf-xarray

But that function only works with points and you have polygons. I am going to define the new lat/lon by getting the exterior points of each polygon. This was helpful: https://gis.stackexchange.com/questions/302430/polygon-to-point-in-geopandas 

In [ ]:
points = emisscheck.copy()

In [ ]:
points.geometry = points.geometry.apply(lambda x: Point(list(x.exterior.coords)))

In [ ]:
fig = plt.figure()
ax = points.plot(column='c3h8_emiss', figsize=(12, 8), cmap='viridis', legend=True)

xlim = ([-105.0, -100.0])
ylim = ([30, 32])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# points.plot(facecolor="none",ax=ax)
p1byp1_grid_gdf.plot(ax=ax, color='red', markersize=2)
plt.title('New emissions map with EPA lat/lon plotted over')
plt.show()

In [ ]:
points.columns

In [ ]:
points_df = pd.DataFrame({'lon': points.geometry.x,
             'lat':points.geometry.y,
             'n_wells':points.n_wells,
             'ch4_kgs':points.ch4_emiss_,
             'ch4_kgs_unc':points.ch4_emis_1,
             'c3h8_kgs':points.c3h8_emiss,
             'c3h8_kgs_unc':points.c3h8_emi_1,
             'c2h6_kgs':points.c2h6_emiss,
             'c2h6_kgs_unc':points.c2h6_emi_1}).set_index(['lat','lon'])

In [ ]:
points_xr = points_df.to_xarray()

In [ ]:
points_xr['ch4_kgs'].plot()

In [ ]:
points_xr.to_netcdf('/Users/arianatribby/git/oklahoma_propane/data/processing/wellhead_emiss_for_footprint.nc4')



Compare EPA to stilt footprint 

In [ ]:
footprints_alldays = xr.concat([xr.open_dataset(f) for f in glob.glob('/Users/arianatribby/git/oklahoma_propane/data/stilt_hpc_downloads/noaa_flasknair2017_2020/*.nc')], dim='time')


In [ ]:
footprints_alldays

In [ ]:
print(max(footprints_alldays.lat.values))
print(min(footprints_alldays.lat.values))
print(max(footprints_alldays.lon.values))
print(min(footprints_alldays.lon.values))


In [ ]:
print(max(points_xr.lat.values))
print(min(points_xr.lat.values))
print(max(points_xr.lon.values))
print(min(points_xr.lon.values))

In [ ]:
emisscheck.total_bounds

In [ ]:
max(footprints_alldays.lat.values - epagrid.lat.values[:-50])

In [ ]:
xarr =  footprints_alldays.isel(time=0)
df = xarr.to_dataframe().reset_index()

foot_gdf = gpd.GeoDataFrame(
    df.foot, geometry=gpd.points_from_xy(df.lon,df.lat))

In [ ]:
fig = plt.figure()
# ax = points.plot(column='c3h8_emiss', figsize=(12, 8), cmap='viridis', legend=True)
ax = foot_gdf.plot(column='foot', figsize=(12,8), cmap='viridis', legend=True)

xlim = ([-105.0, -100.0])
ylim = ([30, 32])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# points.plot(facecolor="none",ax=ax)
p1byp1_grid_gdf.plot(ax=ax, color='red', markersize=2)
plt.title('footprint with EPA lat/lon plotted over')
plt.show()

In [ ]:
gridarea = gridarea.reindex_like(ncscd_d30cm, method='nearest', tolerance=0.01)